In [7]:
import os
import sys; sys.path.append("~/HHbbgg_ETH/Training/python") # to load packages
import IO as io
reload(io)

In [45]:
ntuples = '20170509'
# "%" sign allows to interpret the rest as a system command
%env data=$IO.ldata/$ntuples\n
folders = ! ls $data | sort -t_ -k 3 -n
folders

env: data=$IO.ldata/$ntuples\n


['ls: cannot access $IO.ldata/$ntuples\\n: No such file or directory']

In [28]:
import numpy as np

In [29]:
branch_names = 'leadingJet.Pt(),leadingPhoton.Pt()'.split(",")

In [30]:
branch_names = [c.strip() for c in branch_names]
branch_names = (b.replace(" ", "_") for b in branch_names)
branch_names = list(b.replace("-", "_") for b in branch_names)

In [ ]:
signal = adjust_and_compress().to_hdf('/scratch/musella/qcd_all_br_flt16.hd5','qcd',compression=9,complib='bzip2',mode='w')